In [2]:
import pandas as pd
import numpy as np

# Fungsi untuk membersihkan nama supplier 
def clean_supplier_name(name):
    # Ubah ke huruf besar
    name = str(name).upper()
    # Hilangkan spasi di awal dan akhir
    name = name.strip()
    return name

try:
    # Baca dataset supplier
    supplier_df = pd.read_excel('Data_Supplier_dan_Daerahnya.xlsx')
    
    # Tampilkan daftar kolom untuk debugging
    print("Kolom pada file supplier:")
    print(supplier_df.columns.tolist())
    
    # Verifikasi dan gunakan nama kolom yang benar untuk supplier
    supplier_column = None
    for possible_column in ['SUPPLIER', 'Supplier', 'supplier', 'SUPLIER', 'Suplier', 'suplier']:
        if possible_column in supplier_df.columns:
            supplier_column = possible_column
            break
    
    if supplier_column is None:
        # Jika tidak menemukan kolom supplier yang cocok, gunakan indeks kolom ke-3 (indeks 2)
        if len(supplier_df.columns) > 2:
            supplier_column = supplier_df.columns[2]  # Kolom SUPPLIER (indeks 2)
            print(f"Menggunakan kolom '{supplier_column}' sebagai kolom supplier")
        else:
            print("Struktur kolom tidak sesuai. Menampilkan kolom yang tersedia:")
            print(supplier_df.columns)
            raise Exception("Kolom supplier tidak ditemukan")
    
    # Verifikasi kolom kecamatan
    kecamatan_column = None
    for possible_column in ['Kecamatan', 'KECAMATAN', 'kecamatan']:
        if possible_column in supplier_df.columns:
            kecamatan_column = possible_column
            break
    
    if kecamatan_column is None:
        # Jika tidak menemukan kolom kecamatan yang cocok, gunakan indeks kolom ke-5 (indeks 4)
        if len(supplier_df.columns) > 4:
            kecamatan_column = supplier_df.columns[4]
            print(f"Menggunakan kolom '{kecamatan_column}' sebagai kolom kecamatan")
        else:
            print("Struktur kolom tidak sesuai. Menampilkan kolom yang tersedia:")
            print(supplier_df.columns)
            raise Exception("Kolom kecamatan tidak ditemukan")
    
    # Tampilkan nilai unik di kolom Kecamatan untuk verifikasi
    print("\nNilai unik di kolom Kecamatan:")
    print(supplier_df[kecamatan_column].unique())
    
    # Bersihkan nama supplier pada dataset supplier
    supplier_df['supplier_clean'] = supplier_df[supplier_column].apply(clean_supplier_name)
    
    # Baca dataset timbangan
    timbangan_df = pd.read_excel('datatimbangan2019-2023.xlsx', header=0, skiprows=1)
    
    # Tampilkan daftar kolom untuk debugging
    print("\nKolom pada file timbangan:")
    print(timbangan_df.columns.tolist())
    
    # Verifikasi kolom supplier di dataset timbangan
    timbangan_supplier_column = None
    for possible_column in ['Suplier', 'SUPLIER', 'Supplier', 'SUPPLIER', 'supplier', 'suplier']:
        if possible_column in timbangan_df.columns:
            timbangan_supplier_column = possible_column
            break
    
    if timbangan_supplier_column is None:
        try:
            timbangan_supplier_column = timbangan_df.columns[6]
            print(f"Menggunakan kolom '{timbangan_supplier_column}' sebagai kolom supplier timbangan")
        except IndexError:
            print("Kolom supplier tidak ditemukan. Menampilkan 5 baris pertama data timbangan:")
            print(timbangan_df.head())
            raise Exception("Struktur data tidak sesuai yang diharapkan. Cek kembali format file.")
    
    # Bersihkan nama supplier pada dataset timbangan
    timbangan_df['supplier_clean'] = timbangan_df[timbangan_supplier_column].apply(clean_supplier_name)
    
    # Buat kamus pencarian dari data supplier
    supplier_kecamatan_dict = dict(zip(supplier_df['supplier_clean'], supplier_df[kecamatan_column]))
    
    # Tambahkan data manual untuk supplier yang belum terdeteksi otomatis
    manual_supplier_kecamatan = {
        'DEPO MALUKU': 'Magelang Utara',
        'PT. NAGA PLASTIK': 'Magelang Selatan',
        'SMKN 3 MAGELANG': 'Magelang Tengah',
        'DINAS PERTANIAN MGL': 'Magelang Tengah',
        'KEBON SEMILIR MGL': 'Magelang Selatan',
        'UNTID MAGELANG': 'Magelang Selatan',
        'SMKN 1 MAGELANG': 'Magelang Selatan'
    }

    # Pastikan semua key diubah ke format cleaned
    manual_supplier_kecamatan_cleaned = {clean_supplier_name(k): v for k, v in manual_supplier_kecamatan.items()}

    # Update kamus utama dengan data manual
    supplier_kecamatan_dict.update(manual_supplier_kecamatan_cleaned)
    
    print("\nData manual berhasil ditambahkan ke kamus pencarian.")

    # Fungsi untuk mencari kecamatan berdasarkan supplier
    def get_kecamatan(supplier_name):
        if supplier_name in supplier_kecamatan_dict:
            return supplier_kecamatan_dict[supplier_name]
        
        for key_supplier, kecamatan in supplier_kecamatan_dict.items():
            if key_supplier in supplier_name or supplier_name in key_supplier:
                return kecamatan
        
        return np.nan

    # Tambahkan kolom Kecamatan
    timbangan_df['Kecamatan'] = timbangan_df['supplier_clean'].apply(get_kecamatan)

    # Hapus kolom tambahan
    timbangan_df.drop('supplier_clean', axis=1, inplace=True)

    # Simpan hasil ke file baru
    output_file = 'datatimbangan2019-2023_dengan_kecamatan.xlsx'
    timbangan_df.to_excel(output_file, index=False)

    print("\nProses selesai! Data telah disimpan di '" + output_file + "'")

    # Tampilkan informasi pencocokan
    total_data = len(timbangan_df)
    matched_data = timbangan_df['Kecamatan'].notna().sum()
    print(f"Total data: {total_data}")
    print(f"Data dengan kecamatan terisi: {matched_data} ({matched_data/total_data*100:.2f}%)")
    print(f"Data tanpa kecamatan: {total_data - matched_data} ({(total_data - matched_data)/total_data*100:.2f}%)")

    print("\n=== DATA HASIL PENAMBAHAN KOLOM KECAMATAN ===")
    print(timbangan_df.head(10)[['Kecamatan', timbangan_supplier_column]])

    unmatched = timbangan_df[timbangan_df['Kecamatan'].isna()]
    if not unmatched.empty:
        print("\nSupplier yang tidak ditemukan kecamatannya:")
        unique_unmatched = unmatched[timbangan_supplier_column].unique()
        for i, supplier in enumerate(unique_unmatched[:10]):
            print(f"{i+1}. {supplier}")
        if len(unique_unmatched) > 10:
            print(f"... dan {len(unique_unmatched) - 10} supplier lainnya")

    print("\nFrekuensi data per Kecamatan:")
    kecamatan_counts = timbangan_df['Kecamatan'].value_counts().head(10)
    print(kecamatan_counts)

except FileNotFoundError as e:
    print(f"File tidak ditemukan: {str(e)}")
    print("Pastikan nama file sudah benar dan berada di direktori yang sama.")
except Exception as e:
    print(f"Terjadi kesalahan: {str(e)}")
    import traceback
    traceback.print_exc()

Kolom pada file supplier:
['No', 'Supplier', 'SUPPLIER', 'Kelurahan', 'Kecamatan', 'Kota / Kabupaten', 'Notes']

Nilai unik di kolom Kecamatan:
['Magelang Tengah' 'Magelang Selatan ' 'Magelang Utara'
 'Kecamatan Tegalrejo' '-']

Kolom pada file timbangan:
['Tanggal', 'Hari', 'Bulan', 'Tahun', 'No. Polisi', 'Barang', 'SUPPLIER', 'Netto_kg', 'Jam', 'Sopir', 'Admin']

Data manual berhasil ditambahkan ke kamus pencarian.

Proses selesai! Data telah disimpan di 'datatimbangan2019-2023_dengan_kecamatan.xlsx'
Total data: 62547
Data dengan kecamatan terisi: 62532 (99.98%)
Data tanpa kecamatan: 15 (0.02%)

=== DATA HASIL PENAMBAHAN KOLOM KECAMATAN ===
             Kecamatan               SUPPLIER
0    Magelang Selatan      PASAR REJOWINANGUN
1       Magelang Utara  PERUM DEPKES MAGELANG
2    Magelang Selatan      DEPO TIDAR SELATAN
3  Kecamatan Tegalrejo    SOROBAYAN BANYUURIP
4  Kecamatan Tegalrejo        PASAR TEGALREJO
5  Kecamatan Tegalrejo    SOROBAYAN BANYUURIP
6      Magelang Tengah     